# Tutorial 7: Knowledge Distillation

Training large language models can be computationally expensive. **Knowledge Distillation** is a powerful technique for transferring the capabilities of a large, powerful "teacher" model to a smaller, more efficient "student" model. The student learns to mimic the teacher's behavior, resulting in a compact model that performs tasks far beyond its original capabilities.

This tutorial walks through `examples/08_distillation.py` to demonstrate how to use ToolBrain's streamlined `distill()` method.

## The Concept: Teacher-Student Learning

The process is intuitive:

1.  **The Teacher Demonstrates**: A large, capable model (the teacher) is given a task and a set of tools. It performs the task, generating a high-quality execution trace. This trace is a "perfect" example of how to solve the problem.
2.  **The Student Learns**: A smaller, less-capable model (the student) is then trained on these perfect traces from the teacher using supervised learning. It learns to reproduce the teacher's successful reasoning and tool use.
3.  **The Result**: The student model becomes significantly more capable, having absorbed knowledge from the teacher, while remaining small and efficient.

This is incredibly useful for deploying capable agents in resource-constrained environments or for reducing inference costs.

## The Code: `08_distillation.py`

Let's break down the distillation example script.

In [1]:
from smolagents import tool, TransformersModel, CodeAgent
from toolbrain import create_agent, Brain, reward_tool_execution_success

/home/quan/MyProjects/ToolBrain/ToolBrain/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/quan/MyProjects/ToolBrain/ToolBrain/.venv/lib/python3.12/site-packages/toolbrain/adapters.py:9: UserWarning: WARNING: Unsloth should be imported before transformers, peft to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


### Step 1: Define Tools 


In [2]:
@tool
def add(a: int, b: int) -> int:
    """
    Add two integers.

    Args:
        a (int): First addend.
        b (int): Second addend.

    Returns:
        int: Sum of a and b.
    """
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """
    Multiply two integers.

    Args:
        a (int): First factor.
        b (int): Second factor.

    Returns:
        int: Product of a and b.
    """
    return a * b

@tool
def calculate_power(base: float, exponent: int) -> float:
    """
    Calculate base raised to the power of exponent.

    Args:
        base (float): The base number.
        exponent (int): The exponent.

    Returns:
        float: Result of base^exponent.
    """
    return base ** exponent


### Step 2: Prepare Training Data

In [4]:
training_dataset = [
    {
        "query": "Use the add tool to calculate 15 + 27",
        "gold_answer": "42"
    },
    {
        "query": "What is 8 multiplied by 7? Use the multiply tool",
        "gold_answer": "56"
    },
    {
        "query": "Calculate 2 to the power of 5 using the calculate_power tool",
        "gold_answer": "32"
    },
    {
        "query": "Add 100 and 200 using the add tool",
        "gold_answer": "300"
    }
]



### Step 3: Create the Student Agent

First, we create our student agent. Note that we use a small, efficient base model.

In [6]:
# From examples/08_distillation.py

# The student model is small and fast
print("\nCreating student agent (small model)...")
print("📥 Initializing TransformersModel...")
trainable_model = TransformersModel(model_id="Qwen/Qwen2.5-0.5B-Instruct",
                                    max_new_tokens=128)

print("✅ TransformersModel initialized.")

print("🔧 Creating CodeAgent...")
student_agent = CodeAgent(
    tools=[add, multiply, calculate_power],
    model=trainable_model,
    max_steps=1
)
print("✅ Student agent created.")



Creating student agent (small model)...
📥 Initializing TransformersModel...
✅ TransformersModel initialized.
🔧 Creating CodeAgent...
✅ Student agent created.


### Step 4: Initialize the Brain for the Student

Next, we create a `Brain` instance, as usual, configured for our student agent. We'll plan to use GRPO for further training *after* the distillation is complete.

In [7]:
print("\n🧠 Initializing Brain for the student agent...")

brain = Brain(
    student_agent,                          # Agent instance
    algorithm="GRPO",               # Algorithm choice
    reward_func=reward_tool_execution_success  # Use tool success
)


🧠 Initializing Brain for the student agent...


### Step 5: Run Distillation

This is where the magic happens. We call the `brain.distill()` method, providing our training dataset and specifying the ID of the more powerful teacher model.

In [8]:
# The brain orchestrates the entire distillation process

print("\n" + "=" * 60)
print("🎓 PHASE 1: KNOWLEDGE DISTILLATION")
print("=" * 60)
print("The student will now learn from a larger teacher model...")

# Distill knowledge from teacher to student
# Teacher automatically uses the same tools as the student
brain.distill(
    dataset=training_dataset,
    teacher_model_id="Qwen/Qwen2.5-7B-Instruct"
)



🎓 PHASE 1: KNOWLEDGE DISTILLATION
The student will now learn from a larger teacher model...

🎓 Distillation mode activated
 Creating teacher model (Qwen/Qwen2.5-7B-Instruct)...
 Teacher will use the same 4 tools as student: ['<smolagents.tools.tool.<locals>.SimpleTool object at 0x7e75c584a9c0>', '<smolagents.tools.tool.<locals>.SimpleTool object at 0x7e75c5049d90>', '<smolagents.tools.tool.<locals>.SimpleTool object at 0x7e75c504ab40>', '<smolagents.default_tools.FinalAnswerTool object at 0x7e75c59ad490>']


Loading checkpoint shards: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:06<00:00,  1.51s/it]


✅ Teacher agent created
    Trace 1/5


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Use the add tool to calculate 15 + 27                                                                           │
│                                                                                                                 │
╰─ TransformersModel - Qwen/Qwen2.5-7B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

...

Execution logs:
42

Out: None

[Step 1: Duration 3.12 seconds| Input tokens: 2,066 | Output tokens: 45]

Reached max steps.

[Step 2: Duration 0.71 seconds| Input tokens: 2,279 | Output tokens: 62]

       Reward: 1.000
       ✅ High-quality trace kept
✅ Collected 5 traces, kept 5 high-quality traces (>0.9)
 Teacher model cleared from GPU memory

🎓 Starting distillation training on student model...
   Using 5 high-quality teacher traces
      Processing batch 1/5 (traces 1-1)
      Processing batch 2/5 (traces 2-2)
      Processing batch 3/5 (traces 3-3)
      Processing batch 4/5 (traces 4-4)
      Processing batch 5/5 (traces 5-5)
✅ Distillation complete! Student model pre-trained with teacher knowledge

 Starting regular training with RL...


**What happens inside `brain.distill()`?**

1.  **Teacher Creation**: ToolBrain temporarily creates a new "teacher" agent internally, using the `teacher_model_id` and the *same tools* as the student.
2.  **Trace Generation**: The teacher agent runs through the `dataset`, generating high-quality execution traces. Because the teacher is a powerful model, these traces are expected to be correct and effective.
3.  **Supervised Training**: The `Brain` then uses these teacher-generated traces as a dataset for a supervised fine-tuning run on the *student* model. The student learns to imitate the teacher's output directly.
4.  **Cleanup**: After the training is complete, the large teacher model is cleared from memory.

At the end of this step, our student agent's base model has been fine-tuned with the knowledge from the teacher. It is now significantly smarter.

### Step 6: Continue with Reinforcement Learning

After distillation, the student is already "pre-trained" with good behaviors. We can now continue to fine-tune it even further using reinforcement learning. The RL process will be much more efficient because the agent is starting from a much better baseline.

In [9]:
print("\n" + "=" * 60)
print("PHASE 2: REINFORCEMENT LEARNING TRAINING")
print("=" * 60)
print("Now continuing with regular RL training on the pre-trained student...")

brain.train(training_dataset, num_iterations=2)

print("\n" + "=" * 60)
print("TRAINING COMPLETE!")


PHASE 2: REINFORCEMENT LEARNING TRAINING
Now continuing with regular RL training on the pre-trained student...

🚀 Starting training...

--- Iteration 1/2 ---

🔄 Training step for query: 'Use the add tool to calculate 15 + 27...'
Calling get_trace with query: 'Use the add tool to calculate 15 + 27...'
Enable tool retrieval: False
    📝 Trace 1/2


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Use the add tool to calculate 15 + 27                                                                           │
│                                                                                                                 │
╰─ TransformersModel - Qwen/Qwen2.5-0.5B-Instruct ────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Error in code parsing:
Your code snippet is invalid, because the regex pattern <code>(.*?)</code> was not found in it.
            Here is your code snippet:
            Thought: To solve this task, I need to perform addition operations on the numbers 15 and 27.
Code:
add(15, 27)
</code>
            Make sure to include code with the correct pattern, for instance:
            Thoughts: Your thoughts
            <code>
            # Your python code here
            </code>
Make sure to provide correct code blobs.

[Step 1: Duration 0.70 seconds| Input tokens: 2,066 | Output tokens: 38]

...

Code parsing failed on line 8 due to: SyntaxError
total = add.add(num1, num
                ^
Error: '(' was never closed (<unknown>, line 8)

[Step 1: Duration 1.97 seconds| Input tokens: 2,067 | Output tokens: 128]

Reached max steps.

[Step 2: Duration 1.84 seconds| Input tokens: 2,451 | Output tokens: 256]

  🎯 Computing rewards for 2 traces...
      Using single-trace reward function
      🎯 Trace 1 Reward: 0.000
      🎯 Trace 2 Reward: 0.000
📈 Sliding window avg reward (last 10): 0.6000
⚠️ All rewards are zero in this batch. Skipping training step to save compute.

🎉 Training finished!

TRAINING COMPLETE!


---

Knowledge distillation is a cost-effective strategy for creating high-performing agents. By leveraging a powerful teacher model for a short period, you can imbue a small, efficient student model with advanced reasoning capabilities, making it ready for deployment or further specialized RL training.